In [1]:
# encoding: utf-8
# encoding: iso-8859-1
# encoding: win-1252
import json
import nltk
from nltk.tokenize import TweetTokenizer
from IPython.display import clear_output, display
from nltk.corpus import stopwords
import re
import math
import operator


tknzr = TweetTokenizer()
with open('../inverted_index/inverted_index.json') as f:
    inverted_index = json.load(f)
    

stopwords_en = set(stopwords.words('english'))
stopwords_pt = nltk.corpus.stopwords.words('portuguese')
especial_c = ["-",",",";","'",")","(",":", ".","’","–","","...",
              "eua","estados","unidos","filme","reino","brasil","franca",
              "espanha","unido","japao","alemanha","italia","suecia","canada"]



### Dicionario de titulo, autor e elenco

In [2]:
titulo = {}
direcao = {}
elenco = {}


def converter_string(t):
    t = t.lower()  
    t= t.replace("ã©","é")
    t = t.replace("ã§","ç")
    t= t.replace("ãº","ú")
    t= t.replace("ã¡","á")
    t=t.replace("ã£","ã")
    t=t.replace("ãª","ê")
    t= t.replace("ã³","ó")
    t =t.replace("ãµ","õ")
    t =t.replace("ã´","ô")
    t = t.replace("ã\xad","í")
    t = re.sub('\W+','', t )
    t = ''.join([i for i in t if not i.isdigit()])
    
    return t

nome = ""
inverted_index['titulo.pele']
for obj in inverted_index:
    if("titulo." in obj):
        t = obj
        t = t.replace("titulo.","")
        t = converter_string(t)
        if (t not in stopwords_en) and (t not in stopwords_pt) and (t not in especial_c):
            titulo[t] = inverted_index[obj][1]
    if("direcao." in obj):
        t = obj
        t = t.replace("direcao.","")
        t = converter_string(t)
        if (t not in stopwords_en) and (t not in stopwords_pt) and (t not in especial_c):
            direcao[t] = inverted_index[obj][1]
    if("elenco." in obj):
        t = obj
        t = t.replace("elenco.","")
        t = converter_string(t)
        if (t not in stopwords_en) and (t not in stopwords_pt) and (t not in especial_c):
            elenco[t] = inverted_index[obj][1]

print(len(elenco))
print(len(titulo))
print(len(direcao))

18104
3742
2243


### Dicionario com as probabilidades de cada palavra

In [3]:
prob_titulo = {}
prob_direcao = {}
prob_elenco = {}

qtd_documents = 3892
for key, valor in titulo.items():
    prob_titulo[key] = len(valor)/qtd_documents

qtd_direcao = len(direcao)
for key, valor in direcao.items():
    prob_direcao[key] = len(valor)/qtd_documents

qtd_elenco = len(elenco)
for key, valor in elenco.items():
    prob_elenco[key] = len(valor)/qtd_documents

#prob_direcao

#### Carega o cython

In [4]:
%load_ext Cython

In [32]:
%%cython
from __main__ import *


def mutual_info(dicionario, prob_list):
    retorno = {}
    independente = 1
    cont = 0
    keys = list(dicionario.keys())
    tamanho = len(keys)
    for i in range(tamanho):
        cont = cont+1
        clear_output(wait=True)
        print("porcentagem: " + str(((cont/tamanho)*100)))
        key_a = keys[i]
        val_a = dicionario[key_a]
        for j in range(i+1,tamanho):
            key_b = keys[j]
            val_b = dicionario[key_b]
            for k in range(j+1,tamanho):
                key_c = keys[k]
                val_c = dicionario[key_c]
                for a in val_a:
                    for b in val_b:
                        if b == a:
                            for c in val_c:
                                if c == a:
                                    t = (key_a, key_b, key_c)
                                    #print(t)                    
                                    if t in retorno:
                                        retorno[t] = retorno[t] +1
                                    else:
                                        retorno[t] = 1
                                if c > a:
                                    break
                        if b > a:
                            break
                if k>1000:
                    break
    for n in retorno:
        pn = retorno[n]/tamanho
        retorno[n] = math.log2(pn/(prob_list[n[0]]*prob_list[n[1]]*prob_list[n[2]]))

    return retorno




### Mutual information direção

In [12]:
#direcao_mi = mutual_info(direcao, prob_direcao)
# sorted_direcao = sorted(direcao_mi.items(), key=operator.itemgetter(1), reverse=True)
# sorted_direcao

[(('jee', 'woon', 'yim'), 24.64767407925706),
 (('jee', 'woon', 'pil'), 24.64767407925706),
 (('jee', 'woon', 'sung'), 24.64767407925706),
 (('jee', 'yim', 'pil'), 24.64767407925706),
 (('jee', 'yim', 'sung'), 24.64767407925706),
 (('jee', 'pil', 'sung'), 24.64767407925706),
 (('woon', 'yim', 'pil'), 24.64767407925706),
 (('woon', 'yim', 'sung'), 24.64767407925706),
 (('woon', 'pil', 'sung'), 24.64767407925706),
 (('yim', 'pil', 'sung'), 24.64767407925706),
 (('pollard', 'iain', 'forsyth'), 24.64767407925706),
 (('janus', 'metz', 'pedersen'), 24.64767407925706),
 (('anta', 'nio', 'paes'), 24.64767407925706),
 (('anta', 'nio', 'campos'), 24.64767407925706),
 (('anta', 'paes', 'campos'), 24.64767407925706),
 (('nio', 'paes', 'campos'), 24.64767407925706),
 (('veronika', 'franz', 'severin'), 24.64767407925706),
 (('veronika', 'franz', 'fiala'), 24.64767407925706),
 (('veronika', 'severin', 'fiala'), 24.64767407925706),
 (('franz', 'severin', 'fiala'), 24.64767407925706),
 (('haifaa', 'al'

### Mutual information titulo


In [11]:
#titulo_mi  = mutual_info(titulo, prob_titulo)
# sorted_titulo= sorted(titulo_mi.items(), key=operator.itemgetter(1), reverse=True)
# sorted_titulo
# sorted_titulo

[(('borg', 'mc', 'enroe'), 23.90929414110255),
 (('milha', 'pegar', 'pistola'), 23.90929414110255),
 (('negacio', 'ara', 'bias'), 23.90929414110255),
 (('kurt', 'cobain', 'montage'), 23.90929414110255),
 (('kurt', 'cobain', 'heck'), 23.90929414110255),
 (('kurt', 'montage', 'heck'), 23.90929414110255),
 (('cobain', 'montage', 'heck'), 23.90929414110255),
 (('us', 'marshals', 'federais'), 23.90929414110255),
 (('transilva', 'fa', 'rias'), 23.90929414110255),
 (('pior', 'aluno', 'escola'), 23.90929414110255),
 (('faco', 'ideia', 'fazendo'), 23.90929414110255),
 (('kung', 'fu', 'panda'), 23.90929414110255),
 (('nove', 'cronicas', 'berros'), 23.90929414110255),
 (('canta', 'males', 'espanta'), 23.90929414110255),
 (('vic', 'flo', 'viram'), 23.90929414110255),
 (('vic', 'flo', 'urso'), 23.90929414110255),
 (('vic', 'viram', 'urso'), 23.90929414110255),
 (('flo', 'viram', 'urso'), 23.90929414110255),
 (('gemma', 'bovery', 'imitando'), 23.90929414110255),
 (('menina', 'roubava', 'livros'), 23

### Mutual information elenco


In [31]:
elenco_mi  = mutual_info(elenco, prob_elenco)
sorted_elenco= sorted(elenco_mi.items(), key=operator.itemgetter(1), reverse=True)
sorted_elenco

[(('cazarra', 'vina', 'cius'), 21.634867115076442),
 (('tiedje', 'corona', 'kubota'), 21.634867115076442),
 (('tiedje', 'corona', 'taplitz'), 21.634867115076442),
 (('tiedje', 'corona', 'kotsifakis'), 21.634867115076442),
 (('tiedje', 'corona', 'corea'), 21.634867115076442),
 (('tiedje', 'kubota', 'taplitz'), 21.634867115076442),
 (('tiedje', 'kubota', 'kotsifakis'), 21.634867115076442),
 (('tiedje', 'kubota', 'corea'), 21.634867115076442),
 (('tiedje', 'taplitz', 'kotsifakis'), 21.634867115076442),
 (('tiedje', 'taplitz', 'corea'), 21.634867115076442),
 (('tiedje', 'kotsifakis', 'corea'), 21.634867115076442),
 (('corona', 'kubota', 'taplitz'), 21.634867115076442),
 (('corona', 'kubota', 'kotsifakis'), 21.634867115076442),
 (('corona', 'kubota', 'corea'), 21.634867115076442),
 (('corona', 'taplitz', 'kotsifakis'), 21.634867115076442),
 (('corona', 'taplitz', 'corea'), 21.634867115076442),
 (('corona', 'kotsifakis', 'corea'), 21.634867115076442),
 (('kubota', 'taplitz', 'kotsifakis'), 2